In [ ]:
import os
from os import listdir, system
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
%matplotlib inline

In [ ]:
os.getcwd()

In [ ]:
os.chdir('C:\\Users\\basuc\\5005p')

In [ ]:
data = pd.read_csv('multivariate.csv')
median_age = pd.read_csv('median_age.csv')
auto_cat=pd.read_csv('april_case_class.csv')
#auto_cat=pd.read_csv('total_case_class.csv')

In [ ]:
auto_cat.head()

In [ ]:
data.head()

In [ ]:
data = data.merge(median_age, how = 'left', on='statecode')
data = data.drop(columns = ['total_cases_capita','total_cases_cat','total_covid_cases','april_covid_cases','total_covid_cases','population.1'])


In [ ]:
data.head()

In [ ]:
data = data.merge(auto_cat, how = 'left', on='statecode')
data = data.rename({'cat':'total_cases_cat'}, axis=1)
data=data.sort_values(by='april_cases_capita', ascending=True)
data = data.drop(columns = ['pdf'])

In [ ]:
data

In [ ]:
vbm = pd.read_csv('state_total_vbm.csv')

In [ ]:
vbm

In [ ]:
vbm_hotspot = pd.read_csv('vbm_uk_total.csv')

In [ ]:
data = data.merge(vbm, how='left', on='statecode')
data = data.merge(vbm_hotspot, how='left', on='statecode')

In [ ]:
data = data.rename(columns={"capacity_x": "total_vbm", "capacity_y": "total_hotspot"}, errors="raise")

In [ ]:
data= data[data['total_vbm'].notna()]

In [ ]:
data

In [ ]:
data = data.fillna(0)

In [ ]:
data

In [ ]:
data['total_vbm'] = data['total_vbm']/data['total_vbm'].max()
data['total_hotspot'] = data['total_hotspot']/data['total_hotspot'].max()

In [ ]:
data=data.sort_values(by='april_cases_capita', ascending=True)

In [ ]:
data=data.reset_index(drop=True)

In [ ]:
multivariate = data

In [ ]:
multivariate

In [ ]:
data = data.drop(columns = ['population','density_per_km2','lit_rate','gdp_capita','state'])

In [ ]:
statecode = data['statecode']
data = data.drop('statecode', axis=1)
total_cases_capita = data['april_cases_capita']
data = data.drop('april_cases_capita', axis=1)

In [ ]:
data

In [ ]:
target = data['total_cases_cat']
data = data.drop('total_cases_cat', axis=1)

In [ ]:
data

In [ ]:
from sklearn.decomposition import PCA # Principal Component Analysis module
from sklearn.manifold import TSNE # TSNE module

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
# minmaxscaler = MinMaxScaler()
# data_rescaled = minmaxscaler.fit_transform(data)
stdscaler = StandardScaler()
data_rescaled = stdscaler.fit_transform(data)

In [ ]:
data_rescaled

In [ ]:
#95% of variance
# from sklearn.decomposition import PCA
# pca = PCA(n_components = 2)
# pca.fit(data_rescaled)
# reduced = pca.transform(data_rescaled)

In [ ]:
#95% of variance
from sklearn.decomposition import PCA
pca = PCA().fit(data_rescaled)

%matplotlib inline
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (12,6)

fig, ax = plt.subplots()
xi = np.arange(1,8, step=1)
y = np.cumsum(pca.explained_variance_ratio_)

plt.ylim(0.0,1.1)
plt.plot(xi, y, marker='o', linestyle='--', color='b')

plt.xlabel('Number of Components')
plt.xticks(np.arange(0, 8, step=1)) #change from 0-based array index to 1-based human-readable label
plt.ylabel('Cumulative variance (%)')
plt.title('The number of components needed to explain variance')

plt.axhline(y=0.85, color='r', linestyle='-')
plt.text(0.5, 0.75, '85% cut-off threshold', color = 'red', fontsize=16)

ax.grid(axis='x')
plt.show()

In [ ]:
# Invoke the PCA method on the standardised data
pca = PCA(n_components=2)
pca_3d_std = pca.fit_transform(data_rescaled)
pca.explained_variance_ratio_

In [ ]:
print(abs( pca.components_ ))

In [ ]:
# Invoke the TSNE method
tsne = TSNE(n_components=2, verbose=5, perplexity=3, method= 'exact', n_iter=30000, init='pca')
tsne_results_std = tsne.fit_transform(data_rescaled)

In [ ]:
tsne_results_std

In [ ]:
pca1=pca_3d_std[:,0]
pca2=pca_3d_std[:,1]
# pca3=pca_3d_std[:,2]
tsne1 = tsne_results_std[:,0]
tsne2 = tsne_results_std[:,1]
# tsne3 = tsne_results_std[:,2]

In [ ]:
pca1_df = pd.DataFrame(pca1, columns = ['pca1'])
pca2_df = pd.DataFrame(pca2, columns = ['pca2'])
# pca3_df = pd.DataFrame(pca3, columns = ['pca3'])
tsne1_df = pd.DataFrame(tsne1, columns = ['tsne1'])
tsne2_df = pd.DataFrame(tsne2, columns = ['tsne2'])
# tsne3_df = pd.DataFrame(tsne3, columns = ['tsne3'])

In [ ]:
result = pd.concat([multivariate, pca1_df], axis=1, join="inner")
result = pd.concat([result, pca2_df], axis=1, join="inner")
# result = pd.concat([result, pca3_df], axis=1, join="inner")
result = pd.concat([result, tsne1_df], axis=1, join="inner")
result = pd.concat([result, tsne2_df], axis=1, join="inner")
# result = pd.concat([result, tsne3_df], axis=1, join="inner")

In [ ]:
result

In [ ]:
from plotly.subplots import make_subplots
import plotly.express as px
import plotly.graph_objects as go

In [ ]:
# import plotly.express as px
# fig = px.scatter_3d(result, x="pca1", y="pca2", z='pca3', color="total_cases_cat", 
#                     color_discrete_map={
#                 "Low": "#fecc5c",
#                 "Mid": "#fd8d3c",
#                 "High": "#e31a1c"},
#               size='april_cases_capita', hover_data=['state'] )
# fig.show()

In [ ]:
# import plotly.express as px
# fig = px.scatter_3d(result, x="tsne1", y="tsne2", z='tsne3', color="total_cases_cat", 
#                     color_discrete_map={
#                 "Low": "#fecc5c",
#                 "Mid": "#fd8d3c",
#                 "High": "#e31a1c"},
#               size='april_cases_capita', hover_data=['state'] )
# fig.show()

In [ ]:
import plotly.express as px
fig = px.scatter(result, x="pca1", y="pca2",  color="total_cases_cat", 
                 color_discrete_map={
                "Low": "#fecc5c",
                "Mid": "#fd8d3c",
                "High": "#e31a1c"},
              size='april_cases_capita', hover_data=['state'] )
fig.show()

In [ ]:
import plotly.express as px
#df = px.data.iris()
fig = px.scatter(multivariate, x="april_cases_capita", y="urban_pop_share", color="total_cases_cat",
                 color_discrete_map={
                "Low": "#fecc5c",
                "Mid": "#fd8d3c",
                "High": "#e31a1c"},
                 size='april_cases_capita', hover_data=['state'], facet_col='total_cases_cat',log_x=True)
fig.show()

In [ ]:
import plotly.express as px
fig = px.scatter(multivariate, x="april_cases_capita", y="2018HDI", color="total_cases_cat",
                 color_discrete_map={
                "Low": "#fecc5c",
                "Mid": "#fd8d3c",
                "High": "#e31a1c"},
                 size='april_cases_capita', hover_data=['statecode'], facet_col='total_cases_cat',log_x=True)
fig.show()

In [ ]:
# multivariate['total_vbm'] = (multivariate['total_vbm']-multivariate['total_vbm'].min())/(multivariate['total_vbm'].max()-multivariate['total_vbm'].min())

In [ ]:
# multivariate['total_vbm'] = (multivariate['total_hotspot']-multivariate['total_hotspot'].min())/(multivariate['total_hotspot'].max()-multivariate['total_hotspot'].min())

In [ ]:
# multivariate = multivariate.drop(multivariate[multivariate.statecode=='DL'].index)

In [ ]:
# multivariate = multivariate.drop(multivariate[multivariate.statecode=='MH'].index)

In [ ]:
# multivariate = multivariate.drop(multivariate[multivariate.statecode=='KA'].index)

In [ ]:
multivariate

In [ ]:
import plotly.express as px
fig = px.scatter(multivariate, x="april_cases_capita", y="total_vbm", color="total_cases_cat",
                 color_discrete_map={
                "Low": "#fecc5c",
                "Mid": "#fd8d3c",
                "High": "#e31a1c"},
                 size='april_cases_capita', hover_data=['statecode'], facet_col='total_cases_cat',log_x=True,
                labels={
                     "april_cases_capita": "Infections Per Capita(Apr-21)",
                     "total_vbm": "Total International Arrivals",
                     "total_cases_cat": "Infection Category"  
                 },
                   title = "Total International Arrivals Vs Infection Per Capita Apr-21")
fig.update_layout(
    font_family="Times New Roman",
    font_color="black",
    title_font_family="Times New Roman",
    title_font_color="black",

)
fig.show()
fig.write_html(r"C:\Users\basuc\5005p\figs\international arrival2.html")

In [ ]:
import plotly.express as px
fig = px.scatter(multivariate, x="april_cases_capita", y="total_hotspot", color="total_cases_cat",
                 color_discrete_map={
                "Low": "#fecc5c",
                "Mid": "#fd8d3c",
                "High": "#e31a1c"},
                 size='april_cases_capita', hover_data=['statecode'], facet_col='total_cases_cat',log_x=True)
fig.show()

In [ ]:
import plotly.express as px
#df = px.data.iris()
fig = px.scatter(multivariate, x="urban_pop_share", y="total_hotspot", color="total_cases_cat",
                 color_discrete_map={
                "Low": "#fecc5c",
                "Mid": "#fd8d3c",
                "High": "#e31a1c"},
                 size='april_cases_capita', hover_data=['state']
                 #, facet_col='total_cases_cat',log_x=True
                )
fig.add_hline(y=0.2,line_width=0.5, line_dash="dash",)
fig.add_vline(x=40,line_width=0.5, line_dash="dash",)
fig.show()